In [ ]:
import arcpy
import psycopg2
xys = [[5,5], [10,10], [10,5]]
array = arcpy.Array([arcpy.Point(*xy) for xy in xys])
polygon = arcpy.Polygon(array)

pt = arcpy.Point(1, 2)
pt_geometry = arcpy.PointGeometry(pt)
pt_geometry

xyz = [[1,1], [2,2], [3,3]]
LineArray = arcpy.Array([arcpy.Point(*xy) for xy in xyz])
line = arcpy.Polyline(LineArray)
line

polygon

sr = arcpy.SpatialReference(4326)
polygon.SR = sr
wkt = polygon.WKT

connection = psycopg2.connect(
    host="",
    database="",
    user="",
    password=''
)

cursor = connection.cursor()
table_query = "CREATE TABLE IF NOT EXISTS polygon (objectid SERIAL PRIMARY KEY, geom GEOMETRY)"

cursor.execute(table_query)
connection.commit()

insert_query = f"INSERT INTO polygon (geom) VALUES (ST_GeomFromText('{wkt}', 4326))"
cursor.execute(insert_query)
connection.commit()

del connection
del cursor

In [ ]:
import psycopg2
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
    return "<h1>Hello World</h1>"

@app.route('/polygon')
def polygon():
    connection = psycopg2.connect(
        host="",
        database="",
        user="",
        password=''
    )

    cursor = connection.cursor()
    query = "SELECT JSON_BUILD_OBJECT('type', 'FeatureCollection', 'features', JSON_AGG(ST_AsGeoJSON(polygon.*)::json)) FROM polygon;"

    cursor.execute(query)
    data = cursor.fetchone()[0]

    return data

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)